In [99]:
import functions as fn
import pandas as pd
from importlib import reload
from tqdm import tqdm
from termcolor import colored
import matplotlib.pyplot as plt
from ooipy.tools import ooiplotlib as ooiplt
import datetime

In [107]:
df = pd.read_csv('E:/Acer backup 3/internships and jobs/UW/Reader grader and TA/Shima Abadi/Data/oregon_hydrate.csv',sep = ',')


In [108]:
df.columns

Index(['Unnamed: 0', 'MMSI', 'BaseDateTime', 'LAT', 'LON', 'SOG', 'COG',
       'Heading', 'VesselName', 'IMO', 'CallSign', 'Status', 'LENGTH', 'Width',
       'Draft', 'Cargo', 'TransceiverClass', 'TIMESTAMP UTC',
       'oregon_slope_Lat', 'oregon_slope_Lon', 'oregon_slope_Loc', 'ship_Loc',
       'Southern_hydrate_Lat', 'Southern_hydrate_lon', 'Southern_hydrate_Loc',
       'distance(in km) oregon', 'distance(in km) hydrate', 'Type Code',
       'VESSEL TYPE', 'Group', 'group3'],
      dtype='object')

In [109]:
df['TIMESTAMP UTC']=df['TIMESTAMP UTC'].astype('datetime64[ns]')


In [110]:
df.head()

,Unnamed: 0,MMSI,BaseDateTime,LAT,LON,SOG,COG,Heading,VesselName,IMO,...,ship_Loc,Southern_hydrate_Lat,Southern_hydrate_lon,Southern_hydrate_Loc,distance(in km) oregon,distance(in km) hydrate,Type Code,VESSEL TYPE,Group,group3
0,0,636092819,2021-01-01T00:17:40,44.56820,-125.62248,11.0,180.6,181.0,NaN,NaN,...,"(44.5682, -125.62248)",44.5691,-125.1481,"(44.5691, -125.1481)",19.40,37.68,NaN,NaN,NaN,NaN
1,1,636092819,2021-01-01T00:19:00,44.56414,-125.62273,11.1,187.9,180.0,NaN,NaN,...,"(44.56414, -125.62273)",44.5691,-125.1481,"(44.5691, -125.1481)",19.29,37.71,NaN,NaN,NaN,NaN
2,2,636092819,2021-01-01T00:20:08,44.56068,-125.62271,11.4,162.9,179.0,NaN,NaN,...,"(44.56068, -125.62271)",44.5691,-125.1481,"(44.5691, -125.1481)",19.19,37.72,NaN,NaN,NaN,NaN
3,3,636092819,2021-01-01T00:22:19,44.55397,-125.62281,10.9,186.6,182.0,NaN,NaN,...,"(44.55397, -125.62281)",44.5691,-125.1481,"(44.5691, -125.1481)",19.01,37.75,NaN,NaN,NaN,NaN
4,4,636092819,2021-01-01T00:24:00,44.54881,-125.62310,10.8,177.9,180.0,NaN,NaN,...,"(44.54881, -125.6231)",44.5691,-125.1481,"(44.5691, -125.1481)",18.91,37.81,NaN,NaN,NaN,NaN


In [111]:
df['TIMESTAMP UTC'].min()

Timestamp('2017-01-01 09:48:46')

In [112]:
df['TIMESTAMP UTC'].max()

Timestamp('2022-06-30 23:59:29')

In [113]:
df['VESSEL TYPE'].unique()

array([nan, 'Cargo, all ships of this type', 'Pleasure Craft', 'Fishing',
       'Sailing', 'Passenger, all ships of this type',
       'Tanker, all ships of this type',
       'Other Type, all ships of this type',
       'Cargo, Hazardous category A', 'Towing',
       'Cargo, Hazardous category C',
       'Other Type, no additional information',
       'Cargo, No additional information', 'Spare - Local Vessel',
       'Tanker, No additional information', 'Cargo, Hazardous category B',
       'Cargo, Hazardous category D', 'Tanker, Hazardous category B',
       'Tanker, Hazardous category A', 'Tug', 'Not available (default)',
       'Wing in ground (WIG), all ships of this type',
       'Other Type, Hazardous category A', 'Tanker, Hazardous category D',
       'Military ops',
       'Towing: length exceeds 200m or breadth exceeds 25m',
       'Tanker, Hazardous category C',
       'Passenger, No additional information'], dtype=object)

In [114]:
min_duration=10
path_to_write='E:/Acer backup 3/internships and jobs/UW/Reader grader and TA/Shima Abadi/Data/Benchmark_data/'

In [115]:
reload(fn)

<module 'functions' from 'C:\\Users\\khirod\\Documents\\GitHub\\Ocean-data-analysis\\functions.py'>

In [116]:
def get_benchmark_data(df,hydrophone_idx,inner_rad,outer_rad,min_duration,fmin=10,fmax=90,break_dur=True,get_acoustic=True,get_spectrogram=True):
    
    if hydrophone_idx==1:
        hydro='Oregon_slope'
    elif hydrophone_idx==2:
        hydro='Southern_hydrate'
    suffix=str(inner_rad)+'_'+ str(outer_rad)+'.csv'
    ais=fn.choose_df_slope_hydrate(df,hydrophone_idx)
    isolated_ships=fn.get_isolated_ships(ais,inner_rad,outer_rad,min_duration)
    isolated_ships.to_csv(path_to_write+hydro+'/isolated_ships_' + suffix)
    isolated_ais=fn.isolated_ais(ais,isolated_ships,inner_rad)
    isolated_ais.to_csv(path_to_write+hydro+'/isolated_ais_'+ suffix)
    if break_dur:
        isolated_ais_10m=fn.break_duration(isolated_ships)
        isolated_ais_10m.to_csv(path_to_write+hydro+'/isolated_ais_10m_'+ suffix)
    
    if get_acoustic==True:
        print('Saving wav files')
        for i in tqdm(range(len(isolated_ships))):
            min_time=isolated_ships.start_time[i]
            max_time=isolated_ships.end_time[i]
            hydrophone_data=fn.get_acoustic(hydrophone_idx,min_time,max_time,fmin=fmin,fmax=fmax)
            if hydrophone_data==None:
                # print('data trace is none. Continuing to next')
                pass
            else:
                hydrophone_data.wav_write(filename=path_to_write+hydro+'/Audio'+'/isolated_ships_'+str(inner_rad)+'_'+ str(outer_rad)+'_'+str(i)+'.wav')
    if get_spectrogram==True:
        print('Saving spectrograms')
        for i in tqdm(range(len(isolated_ships))):
            min_time=isolated_ships.start_time[i]
            max_time=isolated_ships.end_time[i]
            spectrogram=fn.get_spectrogram_data(hydrophone_idx,min_time,max_time,fmin=fmin,fmax=fmax)
            if (spectrogram==None):
                # print('data trace is none. Continuing to next')
                pass
            else:
                if (spectrogram.values.shape[0]<2) | (spectrogram.values.shape[1]<2):
                    pass
                else:
                    #spectrogram.visualize(save_spec=True,plot_spec=False,filename=path_to_write+hydro+'/Spectrogram'+'/isolated_ships_'+str(inner_rad)+'_'+ str(outer_rad)+'_'+str(i)+'.png')
                    ooiplt.plot_spectrogram(spectrogram,plot=False,save=True,fmin=fmin,fmax=fmax,filename=path_to_write+hydro+'/Spectrogram'+'/isolated_ships_'+str(inner_rad)+'_'+ str(outer_rad)+'_'+str(i)+'.png')
    return isolated_ships,isolated_ais

## Creating benchmark data for Oregon slope

### inner rad=10, outer_rad=30

In [29]:

d1,d2=get_benchmark_data(df,1,10,30,min_duration,get_acoustic=False,get_spectrogram=False)

 Max distance: 57.89 and Min distance: 0.0


100%|█████████████████████████████████████████████████████████████████████████████| 1105/1105 [00:08<00:00, 124.68it/s]


### Depth=2.9
### Oregon slope= 'HYSB1'

### inner rad=2.9, outer_rad=2.9, outer_rad=5.8

In [117]:
inner_rad= 2.9
outer_rad1=2.9
outer_rad2=5.8

In [118]:
d1,d2=get_benchmark_data(df,1,inner_rad,outer_rad1,min_duration,get_acoustic=False,get_spectrogram=False)

 Max distance: 57.89 and Min distance: 0.0


100%|█████████████████████████████████████████████████████████████████████████████████| 70/70 [00:00<00:00, 106.02it/s]


{'MMSI': 235010450, 'start_time': Timestamp('2017-07-05 06:37:19'), 'end_time': Timestamp('2017-07-05 06:47:19'), 'VESSEL TYPE': nan, 'len_of_recording': 10.0}


In [68]:
d1

,MMSI,start_time,end_time,count,VESSEL TYPE,len_of_recording
0,235010450,2017-05-23 18:40:59,2017-05-23 18:51:59,11,1004.0,11.000000
1,235010450,2017-07-05 06:37:19,2017-07-05 06:47:19,10,1004.0,10.000000
2,246028000,2017-09-27 01:59:36,2017-09-27 02:11:12,10,1012.0,11.600000
3,255805864,2017-01-27 13:58:44,2017-01-27 14:10:01,11,1004.0,11.283333
4,255805864,2017-04-01 23:34:25,2017-04-01 23:49:03,11,1004.0,14.633333
...,...,...,...,...,...,...
65,477280800,2017-10-03 10:15:33,2017-10-03 10:29:32,9,70.0,13.983333
66,477280800,2017-10-27 08:04:16,2017-10-27 08:18:06,8,70.0,13.833333
67,538008492,2022-03-08 06:04:29,2022-03-08 06:16:36,12,70.0,12.116667
68,538071203,2021-11-13 13:08:47,2021-11-13 13:21:19,9,36.0,12.533333


In [119]:
print(d1['VESSEL TYPE'].value_counts(),'\n',d1['len_of_recording'].sum())

Other Type, all ships of this type    16
Towing                                 5
Sailing                                3
Cargo, all ships of this type          3
Fishing                                2
Pleasure Craft                         1
Name: VESSEL TYPE, dtype: int64 
 28560.516666666663


In [120]:
d1,d2=get_benchmark_data(df,1,inner_rad,outer_rad2,min_duration,get_acoustic=False,get_spectrogram=False)

 Max distance: 57.89 and Min distance: 0.0


100%|█████████████████████████████████████████████████████████████████████████████████| 84/84 [00:00<00:00, 125.53it/s]


{'MMSI': 235010450, 'start_time': Timestamp('2017-07-05 06:37:19'), 'end_time': Timestamp('2017-07-05 06:47:19'), 'VESSEL TYPE': nan, 'len_of_recording': 10.0}


In [121]:
print(d1['VESSEL TYPE'].value_counts(),'\n',d1['len_of_recording'].sum())

Other Type, all ships of this type    22
Towing                                 5
Sailing                                3
Cargo, all ships of this type          3
Fishing                                2
Pleasure Craft                         1
Name: VESSEL TYPE, dtype: int64 
 26752.316666666666


In [58]:
d1,d2=get_benchmark_data(df,1,inner_rad,outer_rad2,min_duration,get_acoustic=False,get_spectrogram=False))

 Max distance: 57.89 and Min distance: 0.0


100%|█████████████████████████████████████████████████████████████████████████████████| 84/84 [00:00<00:00, 128.32it/s]


{'MMSI': 235010450, 'start_time': Timestamp('2017-07-05 06:37:19'), 'end_time': Timestamp('2017-07-05 06:47:19'), 'VESSEL TYPE': 1004.0, 'len_of_recording': 10.0}
Saving spectrograms


  0%|                                                                                           | 0/84 [00:00<?, ?it/s]

OO.AXBA1.AXBA1.HDH | 2017-05-23T18:40:59.000000Z - 2017-05-23T18:50:59.000000Z | 200.0 Hz, 120001 samples


...

 98%|████████████████████████████████████████████████████████████████████████████████  | 82/84 [03:49<00:06,  3.00s/it]

OO.AXBA1.AXBA1.HDH | 2021-11-13T13:08:47.000000Z - 2021-11-13T13:18:47.000000Z | 200.0 Hz, 120001 samples


 99%|█████████████████████████████████████████████████████████████████████████████████ | 83/84 [03:51<00:02,  2.86s/it]

OO.AXBA1.AXBA1.HDH | 2017-08-30T20:24:33.000000Z - 2017-08-30T20:34:33.000000Z | 200.0 Hz, 120001 samples


100%|██████████████████████████████████████████████████████████████████████████████████| 84/84 [03:53<00:00,  2.78s/it]


In [101]:
d1.head()

,MMSI,time_min,time_max,count,VESSEL TYPE,len_of_recording
0,210059000,2016-11-25 22:49:56,2016-11-25 23:02:56,2,Bulk Carrier,13.000000
1,212250000,2016-04-01 18:58:25,2016-04-01 19:22:08,3,Bulk Carrier,23.716667
2,212482000,2020-09-24 19:41:45,2020-09-24 19:57:23,5,Bulk Carrier,15.633333
3,220518000,2018-01-20 21:28:28,2018-01-20 22:07:29,3,Oil Products Tanker,39.016667
4,249666000,2019-04-19 04:21:58,2019-04-19 04:36:30,6,Passenger Ship,14.533333


## Creating benchmark data for Southern hydrate

### Depth=772
### Oregon slope= 'HYS14'

### inner rad=0.772, outer_rad=0.772, outer_rad=1.544

In [122]:
inner_rad= 0.772
outer_rad1=0.772
outer_rad2= 1.544

In [123]:
d1,d2=get_benchmark_data(df,2,inner_rad,outer_rad1,min_duration,get_acoustic=False,get_spectrogram=False)

 Max distance: 76.26 and Min distance: 0.0


100%|█████████████████████████████████████████████████████████████████████████████████| 42/42 [00:00<00:00, 129.51it/s]


In [124]:
print(d1['VESSEL TYPE'].value_counts(),'\n',d1['len_of_recording'].sum())

Other Type, all ships of this type    13
Name: VESSEL TYPE, dtype: int64 
 26602.61666666667


In [125]:
d1,d2=get_benchmark_data(df,2,inner_rad,outer_rad2,min_duration,get_acoustic=False,get_spectrogram=False)

 Max distance: 76.26 and Min distance: 0.0


100%|█████████████████████████████████████████████████████████████████████████████████| 43/43 [00:00<00:00, 128.25it/s]


{'MMSI': 319005600, 'start_time': Timestamp('2018-08-05 02:39:55'), 'end_time': Timestamp('2018-08-05 02:49:55'), 'VESSEL TYPE': nan, 'len_of_recording': 10.0}


In [126]:
print(d1['VESSEL TYPE'].value_counts(),'\n',d1['len_of_recording'].sum())

Other Type, all ships of this type    13
Name: VESSEL TYPE, dtype: int64 
 26585.95


In [53]:
if (spectrogram.values.shape[0]<2) | (spectrogram.values.shape[1]<2):
    pass
else:
    ooiplt.plot_spectrogram(spectrogram,plot=False,save=True,fmin=fmin,fmax=fmax,filename='test.png')

In [35]:
# from functions import choose_df
# ais=choose_df_slope_hydrate(df,1)
# min_duration=20
# lat=float(ais['axial_Loc'].iloc[0][1:8])
# lon=float(ais['axial_Loc'].iloc[0][9:18])
# reload(fn)
# fn.get_isolated_map_plot('scatter',ais,10,30,lat,lon,min_duration)

In [36]:
# reload(fn)
# fn.get_single_isolated_map_plot(1,'scatter',ais,10,30,lat,lon,min_duration,10,90)

## Comparison of various options

In [60]:
inner_rad=[0.772,2.9]
outer_rad=[0.772,1.544,2.9,5.8]
min_duration=10


In [62]:
reload(fn)

print(colored('Comparative view with minimum duration: '+ str(min_duration),'green',attrs=['bold']))
table=pd.DataFrame(columns=['hydrophone_idx','inner_rad','outer_rad','no_unique_ships','no_unique_vessels','no_records'])
for i in tqdm(range(1,3,1)):
    for i_r in tqdm(inner_rad):
        for o_r in outer_rad:
            if i_r<=o_r:
                temp_dict=dict()
                ais=fn.choose_df_slope_hydrate(df,i,verbose=False)

                temp_df1=fn.get_isolated_ships(ais,i_r,o_r,min_duration)
                temp_df2=fn.break_duration(temp_df1)

                temp_dict['hydrophone_idx']=i
                temp_dict['inner_rad']=i_r
                temp_dict['outer_rad']=o_r
                temp_dict['no_unique_ships']=temp_df2['MMSI'].nunique()
                temp_dict['no_unique_vessels']=temp_df2['VESSEL TYPE'].nunique()
                temp_dict['no_records']=len(temp_df2)
                temp_dict['len_of_recordings']=temp_df2.len_of_recording.sum()
                table=table.append(temp_dict,ignore_index=True)
         
table


Comparative view with minimum duration: 10


 50%|██████████████████████████████████████████                                          | 1/2 [00:03<00:03,  3.10s/it]

{'MMSI': 235010450, 'start_time': Timestamp('2017-07-05 06:37:19'), 'end_time': Timestamp('2017-07-05 06:47:19'), 'VESSEL TYPE': 1004.0, 'len_of_recording': 10.0}
{'MMSI': 235010450, 'start_time': Timestamp('2017-07-05 06:37:19'), 'end_time': Timestamp('2017-07-05 06:47:19'), 'VESSEL TYPE': 1004.0, 'len_of_recording': 10.0}



  0%|                                                                                            | 0/2 [00:00<?, ?it/s]

{'MMSI': 319005600, 'start_time': Timestamp('2018-08-05 02:39:55'), 'end_time': Timestamp('2018-08-05 02:49:55'), 'VESSEL TYPE': 1020.0, 'len_of_recording': 10.0}
{'MMSI': 319005600, 'start_time': Timestamp('2018-08-05 02:39:55'), 'end_time': Timestamp('2018-08-05 02:49:55'), 'VESSEL TYPE': 1020.0, 'len_of_recording': 10.0}
{'MMSI': 319005600, 'start_time': Timestamp('2018-08-05 02:39:55'), 'end_time': Timestamp('2018-08-05 02:49:55'), 'VESSEL TYPE': 1020.0, 'len_of_recording': 10.0}



100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:41<00:00, 20.66s/it]


,hydrophone_idx,inner_rad,outer_rad,no_unique_ships,no_unique_vessels,no_records,len_of_recordings
0,1.0,0.772,0.772,4.0,2.0,389.0,3805.300000
1,1.0,0.772,1.544,4.0,2.0,389.0,3805.300000
2,1.0,0.772,2.900,4.0,2.0,389.0,3805.300000
3,1.0,0.772,5.800,4.0,2.0,386.0,3778.450000
4,1.0,2.900,2.900,35.0,14.0,2894.0,28560.516667
5,1.0,2.900,5.800,35.0,14.0,2719.0,26752.316667
6,2.0,0.772,0.772,4.0,2.0,2683.0,26602.616667
7,2.0,0.772,1.544,4.0,2.0,2682.0,26585.950000
8,2.0,0.772,2.900,4.0,2.0,2676.0,26503.966667
9,2.0,0.772,5.800,4.0,2.0,2536.0,24987.833333


In [22]:
reload(fn)

print(colored('Overlapping of all hydrophones with minimum number of timestamps: '+ str(min_duration),'green',attrs=['bold']))
table2=pd.DataFrame(columns=['inner_rad','outer_rad','no_unique_ships','no_unique_vessels','no_records'])

for i_r in inner_rad:
    for o_r in outer_rad:
        combined_df=pd.DataFrame()
        for i in range(1,4,1):
            temp_dict=dict()
            ais=fn.choose_df(df,i,verbose=False)
            d1=fn.get_isolated_ships(ais,i_r,o_r,min_duration)
            combined_df=pd.concat([combined_df,d1])

        temp_dict['inner_rad']=i_r
        temp_dict['outer_rad']=o_r
        temp_dict['no_unique_ships']=combined_df['MMSI'].nunique()
        temp_dict['no_unique_vessels']=combined_df['VESSEL TYPE'].nunique()
        temp_dict['no_records']=len(combined_df)
        table2=table2.append(temp_dict,ignore_index=True)
table2
        

Overlapping of all hydrophones with minimum number of timestamps: 20


,inner_rad,outer_rad,no_unique_ships,no_unique_vessels,no_records
0,5,20,88,19,524
1,5,30,82,19,558
2,5,40,73,19,613
3,10,20,478,37,1374
4,10,30,447,36,1362
5,10,40,396,36,1341
6,15,20,839,42,2683
7,15,30,797,42,2607
8,15,40,718,41,2440


In [ ]:
reload(fn)
df['TIMESTAMP UTC']=df['TIMESTAMP UTC'].astype('datetime64[ns]')
ais = fn.choose_df(df,1)
pings= fn.ais_ping_distribution(ais,show=False)